# Day 24: Lobby Layout

[*Advent of Code 2020 day 24*](https://adventofcode.com/2020/day/24) and [*solution megathread*](https://redd.it/kj96iw)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2020/24/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2020%2F24%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')
import common

downloaded = common.refresh()
%store downloaded >downloaded

Writing 'downloaded' (dict) to file 'downloaded'.


## Part One

In [2]:
HTML(downloaded['part1'])

## Boilerplate

Let's try using [pycodestyle_magic](https://github.com/mattijn/pycodestyle_magic) with pycodestyle (flake8 stopped working for me in VS Code Jupyter). Now how does type checking work?

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [5]:
testdata = """sesenwnenenewseeswwswswwnenewsewsw
neeenesenwnwwswnenewnwwsewnenwseswesw
seswneswswsenwwnwse
nwnwneseeswswnenewneswwnewseswneseene
swweswneswnenwsewnwneneseenw
eesenwseswswnenwswnwnwsewwnwsene
sewnenenenesenwsewnenwwwse
wenwwweseeeweswwwnwwe
wsweesenenewnwwnwsenewsenwwsesesenwne
neeswseenwwswnwswswnw
nenwswwsewswnenenewsenwsenwnesesenew
enewnwewneswsewnwswenweswnenwsenwsw
sweneswneswneneenwnewenewwneswswnese
swwesenesewenwneswnwwneseswwne
enesenwswwswneneswsenwnewswseenwsese
wnwnesenesenenwwnenwsewesewsesesew
nenewswnwewswnenesenwnesewesw
eneswnwswnwsenenwnwnwwseeswneewsenese
neswnwewnwnwseenwseesewsenwsweewe
wseweeenwnesenwwwswnew""".splitlines()

inputdata = downloaded['input'].splitlines()

In [6]:
from enum import Enum


class Direction(Enum):
    def __repr__(self):
        return self.name

    E = (+1, -1, 0)
    NE = (+1, 0, -1)
    NW = (0, +1, -1)
    W = (-1, +1, 0)
    SW = (-1, 0, +1)
    SE = (0, -1, +1)


def parse(steps):
    if len(steps) == 0:
        return
    if steps[0] in {'s', 'n'} and len(steps) > 2:
        return [Direction[steps[0:2].upper()]] + parse(steps[2:])
    elif steps[0] in {'e', 'w'} and len(steps) > 1:
        return [Direction[steps[0].upper()]] + parse(steps[1:])
    else:
        return [Direction[steps.upper()]]


class CubeCoordinate(object):
    def __init__(self, cube):
        (self.x, self.y, self.z) = cube

    def __repr__(self):
        return f'({self.x}, {self.y}, {self.z})'

    def value(self):
        return (self.x, self.y, self.z)

    def to_oddr(self):
        col = self.x + (self.z - (self.z & 1)) // 2
        row = self.z
        return (col, row)

    def from_oddr(hex):
        (hex_col, hex_row) = hex
        cube_x = hex_col - (hex_row - (hex_row & 1)) // 2
        cube_z = hex_row
        cube_y = -cube_x - cube_z
        return CubeCoordinate((cube_x, cube_y, cube_z))

    def add(self, cube):
        (cube_x, cube_y, cube_z) = cube
        cube_sum = (self.x + cube_x, self.y + cube_y, self.z + cube_z)
        return CubeCoordinate(cube_sum)

    def from_directions(self, dirs):
        next_coord = self.add(dirs[0].value)
        if len(dirs) > 1:
            return next_coord.from_directions(dirs[1:])
        else:
            return next_coord


class HexMatrix(object):
    def __init__(self):
        self.tiles = [[True]]
        self.radius = 0
        self.origin = CubeCoordinate.from_oddr((0, 0))
        self.black_adjacent = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

    def __repr__(self):
        output = []
        for i, line in enumerate(self.tiles):
            line_str = ['O' if t else 'X' for t in line]
            if i & 1:
                output.append(' ' + ' '.join(line_str))
            else:
                output.append(' '.join(line_str))
        return '\n'.join(output)

    def repr_black_adjacent(self):
        output = []
        for i, line in enumerate(self.black_adjacent):
            line_str = ['-' if not t else str(t) for t in line]
            if not i & 1:
                output.append(' ' + ' '.join(line_str))
            else:
                output.append(' '.join(line_str))
        return '\n'.join(output)

    def double(self):
        # print(f'doubling: {self.radius}')
        if self.radius == 0:
            new_radius = 1
        else:
            new_radius = self.radius * 2

        self.origin = CubeCoordinate.from_oddr((new_radius, new_radius))

        new_tiles = [[True
                      for x in range(new_radius * 2 + 1)]
                     for y in range(new_radius * 2 + 1)]

        for row in range(len(self.tiles)):
            new_tiles[
                self.radius + row][
                self.radius:(self.radius * 3 + 1)
                ] = self.tiles[row]

        self.radius = new_radius
        self.tiles = new_tiles

    def flip(self, cube):
        # print(f'flipping: {cube}')
        cube_radius = max(map(abs, (cube.x, cube.y, cube.z)))
        while cube_radius >= self.radius:
            self.double()

        (col, row) = self.origin.add(cube.value()).to_oddr()

        self.tiles[row][col] ^= True
        # print(f'flipped: {cube}\n{self}')

    def flip_hex(self, hex):
        (col, row) = hex
        # (o_col, o_row) = self.origin.to_oddr()
        # self.flip(CubeCoordinate.from_oddr((col - o_col, row - o_row)))
        self.flip(CubeCoordinate.from_oddr((col, row)))

    def update_black_adjacent(self):
        adjacent_oddr = [(0, -1), (+1, -1),
                         (-1, 0), (+1, 0),
                         (0, +1), (+1, +1)]
        adjacent_evenr = [(-1, -1), (0, -1),
                          (-1, 0), (+1, 0),
                          (-1, +1), (0, +1)]

        self.black_adjacent = [[0
                                for x in range(len(self.tiles) + 2)]
                               for y in range(len(self.tiles) + 2)]
        for row in range(len(self.tiles)):
            for col in range(len(self.tiles[0])):
                if not self.tiles[row][col]:
                    if row & 1:
                        adjacent = adjacent_oddr
                    else:
                        adjacent = adjacent_evenr

                    for adj_col, adj_row in adjacent:
                        self.black_adjacent[
                            row + adj_row + 1][
                            col + adj_col + 1] += 1

    def next_day(self):
        self.update_black_adjacent()
        # print(self.repr_black_adjacent())
        previous_tiles = [line[:] for line in self.tiles]
        (previous_o_col, previous_o_row) = self.origin.to_oddr()
        for col in range(len(self.black_adjacent[0])):
            if self.black_adjacent[0][col] == 2:
                self.flip_hex((col - 1 - previous_o_col, -1 - previous_o_row))
            if self.black_adjacent[-1][col] == 2:
                self.flip_hex((col - 1 - previous_o_col,
                               len(self.black_adjacent) - previous_o_row))
        for row in range(1, len(self.black_adjacent) - 1):
            if self.black_adjacent[row][0] == 2:
                self.flip_hex((-1 - previous_o_col,
                               row - 1 - previous_o_row))
            if self.black_adjacent[row][-1] == 2:
                self.flip_hex((len(self.black_adjacent[0]) - previous_o_col,
                               row - 1 - previous_o_row))

        for row in range(1, len(self.black_adjacent) - 1):
            for col in range(1, len(self.black_adjacent[0]) - 1):
                if not previous_tiles[row - 1][col - 1]:
                    b_adjacent = self.black_adjacent[row][col]
                    if b_adjacent == 0 or b_adjacent > 2:
                        # print(f'flipping black row {row - 1}'
                        #       f'column {col - 1}')
                        self.flip_hex((col - 1 - previous_o_col,
                                       row - 1 - previous_o_row))
                else:
                    if self.black_adjacent[row][col] == 2:
                        # print(f'flipping white row {row - 1}'
                        #       f'column {col - 1}')
                        self.flip_hex((col - 1 - previous_o_col,
                                       row - 1 - previous_o_row))

### Debugging...

In [7]:
a = HexMatrix()
a.tiles[0][0] = False
print(a)
a.double()
print()
a.tiles = [[False, False, False], [False, True, False], [False, False, False]]
print(a)
a.double()
print()
print(a)
a.double()
print()
print(a)
a.double()
print()
print(a)

X

X X X
 X O X
X X X

O O O O O
 O X X X O
O X O X O
 O X X X O
O O O O O

O O O O O O O O O
 O O O O O O O O O
O O O O O O O O O
 O O O X X X O O O
O O O X O X O O O
 O O O X X X O O O
O O O O O O O O O
 O O O O O O O O O
O O O O O O O O O

O O O O O O O O O O O O O O O O O
 O O O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O O O
 O O O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O O O
 O O O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O O O
 O O O O O O O X X X O O O O O O O
O O O O O O O X O X O O O O O O O
 O O O O O O O X X X O O O O O O O
O O O O O O O O O O O O O O O O O
 O O O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O O O
 O O O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O O O
 O O O O O O O O O O O O O O O O O
O O O O O O O O O O O O O O O O O


In [8]:
print(CubeCoordinate(Direction['NE'].value).add(Direction['SE'].value))

(1, -1, 0)


In [9]:
print(testdata[0])
directions = parse('nwwswee')
origin = CubeCoordinate((0, 0, 0))
print(origin.from_directions(directions))

sesenwnenenewseeswwswswwnenewsewsw
(0, 0, 0)


In [10]:
origin = CubeCoordinate((0, 0, 0))
tile_floor = HexMatrix()
for direction_str in testdata:
    directions = parse(direction_str)
    cube = origin.from_directions(directions)
    tile_floor.flip(cube)

print(len(tile_floor.tiles)**2 - sum(map(sum, tile_floor.tiles)))

10


In [11]:
origin = CubeCoordinate((0, 0, 0))
tile_floor = HexMatrix()
for direction_str in inputdata:
    directions = parse(direction_str)
    cube = origin.from_directions(directions)
    tile_floor.flip(cube)

print(len(tile_floor.tiles)**2
      - sum(map(sum, tile_floor.tiles)))

269


In [12]:
HTML(downloaded['part1_footer'])

## Part Two

In [13]:
HTML(downloaded['part2'])

In [14]:
origin = CubeCoordinate((0, 0, 0))
tile_floor = HexMatrix()
for direction_str in testdata:
    directions = parse(direction_str)
    cube = origin.from_directions(directions)
    tile_floor.flip(cube)

for day in range(1, 101):
    tile_floor.next_day()
    count = (len(tile_floor.tiles)**2
             - sum(map(sum, tile_floor.tiles)))
    print(f'Day {day}: {count}')
    # print(tile_floor)

Day 1: 15
Day 2: 12
Day 3: 25
Day 4: 14
Day 5: 23
Day 6: 28
Day 7: 41
Day 8: 37
Day 9: 49
Day 10: 37
Day 11: 55
Day 12: 54
Day 13: 69
Day 14: 73
Day 15: 84
Day 16: 92
Day 17: 88
Day 18: 107
Day 19: 113
Day 20: 132
Day 21: 133
Day 22: 147
Day 23: 134
Day 24: 177
Day 25: 170
Day 26: 176
Day 27: 221
Day 28: 208
Day 29: 207
Day 30: 259
Day 31: 277
Day 32: 283
Day 33: 270
Day 34: 324
Day 35: 326
Day 36: 333
Day 37: 345
Day 38: 371
Day 39: 380
Day 40: 406
Day 41: 439
Day 42: 466
Day 43: 449
Day 44: 478
Day 45: 529
Day 46: 525
Day 47: 570
Day 48: 588
Day 49: 576
Day 50: 566
Day 51: 636
Day 52: 601
Day 53: 667
Day 54: 672
Day 55: 735
Day 56: 766
Day 57: 723
Day 58: 755
Day 59: 805
Day 60: 788
Day 61: 844
Day 62: 875
Day 63: 908
Day 64: 936
Day 65: 994
Day 66: 943
Day 67: 1015
Day 68: 1029
Day 69: 1058
Day 70: 1106
Day 71: 1158
Day 72: 1146
Day 73: 1125
Day 74: 1159
Day 75: 1202
Day 76: 1344
Day 77: 1277
Day 78: 1345
Day 79: 1320
Day 80: 1373
Day 81: 1420
Day 82: 1431
Day 83: 1469
Day 84: 1561


In [15]:
origin = CubeCoordinate((0, 0, 0))
tile_floor = HexMatrix()
for direction_str in inputdata:
    directions = parse(direction_str)
    cube = origin.from_directions(directions)
    tile_floor.flip(cube)

for day in range(1, 101):
    tile_floor.next_day()
    count = (len(tile_floor.tiles)**2
             - sum(map(sum, tile_floor.tiles)))
    print(f'Day {day}: {count}')
    # print(tile_floor)

Day 1: 258
Day 2: 254
Day 3: 269
Day 4: 301
Day 5: 329
Day 6: 315
Day 7: 336
Day 8: 314
Day 9: 391
Day 10: 347
Day 11: 409
Day 12: 417
Day 13: 460
Day 14: 427
Day 15: 496
Day 16: 479
Day 17: 465
Day 18: 552
Day 19: 543
Day 20: 553
Day 21: 599
Day 22: 623
Day 23: 617
Day 24: 696
Day 25: 711
Day 26: 698
Day 27: 734
Day 28: 725
Day 29: 786
Day 30: 819
Day 31: 854
Day 32: 844
Day 33: 924
Day 34: 896
Day 35: 928
Day 36: 970
Day 37: 1050
Day 38: 1035
Day 39: 1114
Day 40: 1108
Day 41: 1126
Day 42: 1187
Day 43: 1147
Day 44: 1230
Day 45: 1247
Day 46: 1265
Day 47: 1353
Day 48: 1369
Day 49: 1350
Day 50: 1413
Day 51: 1437
Day 52: 1460
Day 53: 1575
Day 54: 1528
Day 55: 1624
Day 56: 1633
Day 57: 1679
Day 58: 1739
Day 59: 1802
Day 60: 1730
Day 61: 1848
Day 62: 1818
Day 63: 1925
Day 64: 1849
Day 65: 1915
Day 66: 2007
Day 67: 2098
Day 68: 2045
Day 69: 2160
Day 70: 2201
Day 71: 2344
Day 72: 2304
Day 73: 2413
Day 74: 2355
Day 75: 2524
Day 76: 2389
Day 77: 2486
Day 78: 2637
Day 79: 2612
Day 80: 2646
Day 8

In [16]:
HTML(downloaded['part2_footer'])